# Machine Translation using OpenNMT

# Jamo and Char Tokenization

In [2]:
cd /home/ubuntu/MT

/home/ubuntu/MT


In [3]:
!pip install hgtk

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6689 sha256=c79e1449aa15b61035cead5f6ce27dcce6f4dc168f720c8444872605bbc5f22e
  Stored in directory: /home/ubuntu/.cache/pip/wheels/96/10/4b/0887dcaf5e0b0d26b807fef3497703fea2c44a76f91d198bd3
Successfully built hgtk


In [4]:
!python OpenNMT-py/tools/apply_morph_ko.py -i OpenNMT-py/Data/src-train.txt -o OpenNMT-py/Data/src-train-morph.txt
!python OpenNMT-py/tools/apply_morph_ko.py -i OpenNMT-py/Data/src-valid.txt -o OpenNMT-py/Data/src-val-morph.txt
!python OpenNMT-py/tools/apply_morph_ko.py -i OpenNMT-py/Data/src-test.txt -o OpenNMT-py/Data/src-test-morph.txt

In [50]:
!python OpenNMT-py/tools/apply_char.py -i OpenNMT-py/Data/tgt-train.txt -o OpenNMT-py/Data/tgt-train-char.txt
!python OpenNMT-py/tools/apply_char.py -i OpenNMT-py/Data/tgt-valid.txt -o OpenNMT-py/Data/tgt-val-char.txt
!python OpenNMT-py/tools/apply_char.py -i OpenNMT-py/Data/tgt-test.txt -o OpenNMT-py/Data/tgt-test-char.txt

# **Preprocess the data**

We will be working with some example data in data/ folder.

The data consists of parallel source (src) and target (tgt) data containing one sentence per line with tokens separated by a space:

1. src-train.txt

2. tgt-train.txt

3. src-val.txt

4. tgt-val.txt


Train data and validataion data are required for machine translation training.

Validation files are required and used to evaluate the convergence of the training. It usually contains no more than 5000 sentences.


> If you think about it briefly, you can specify the path of train data and validation data, and specify the path and name to save in -save_data.

> If you want to set vocab size add below command
<br>
-src_vocab_size 32000 -tgt_vocab_size 32000

The vocab size is usually 32000.

In [1]:
!python OpenNMT-py/preprocess.py -train_src OpenNMT-py/Data/src-train-morph.txt -train_tgt OpenNMT-py/Data/tgt-train-morph.txt -valid_src OpenNMT-py/Data/src-val-morph.txt -valid_tgt OpenNMT-py/Data/tgt-val-morph.txt -save_data OpenNMT-py/Data/morph2morph/demo -src_vocab_size -1 -tgt_vocab_size -1 -overwrite

[2020-12-08 02:32:13,402 INFO] Extracting features...
[2020-12-08 02:32:13,404 INFO]  * number of source features: 0.
[2020-12-08 02:32:13,404 INFO]  * number of target features: 0.
[2020-12-08 02:32:13,404 INFO] Building `Fields` object...
[2020-12-08 02:32:13,404 INFO] Building & saving training data...
[2020-12-08 02:32:21,124 INFO] Building shard 0.
[2020-12-08 02:33:09,868 INFO]  * saving 0th train data shard to OpenNMT-py/Data/morph2morph/demo.train.0.pt.
[2020-12-08 02:33:45,829 INFO]  * tgt vocab size: 161817.
[2020-12-08 02:33:48,395 INFO]  * src vocab size: 692838.
[2020-12-08 02:33:53,091 INFO] Building & saving validation data...
[2020-12-08 02:33:56,839 INFO] Building shard 0.
[2020-12-08 02:33:57,338 INFO]  * saving 0th valid data shard to OpenNMT-py/Data/morph2morph/demo.valid.0.pt.


!python OpenNMT-py/preprocess.py -train_src OpenNMT-py/Data/src-train-bpe.txt -train_tgt OpenNMT-py/Data/tgt-train-bpe.txt -valid_src OpenNMT-py/Data/src-val-bpe.txt -valid_tgt OpenNMT-py/Data/tgt-val-bpe.txt -save_data OpenNMT-py/Data/demo -src_vocab_size 32000 -tgt_vocab_size 32000

# **Train the data(Transformer)**

https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf


> If you get GPU-related errors, try halving batch_size

**Below is the full command, and if you want to know more about it, search about Transformer.**

!python OpenNMT-py/train.py -data OpenNMT-py/data/demo -save_model OpenNMT-py/data/model/model -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 -world_size 1 -gpu_rank 0  

In [2]:
!nvidia-smi

Tue Dec  8 02:33:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:00:06.0 Off |                    0 |
| N/A   26C    P8    13W / 250W |      0MiB / 22698MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [9]:
!python OpenNMT-py/train.py -data OpenNMT-py/Data/morph2morph/demo -save_model OpenNMT-py/Data/morph2morph/model/model -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 50000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 10000 -world_size 1 -gpu_rank 0 -tensorboard -tensorboard_log_dir 'logs/my_board/morph2morph'

[2020-12-09 00:10:53,989 INFO]  * src vocab size = 692838
[2020-12-09 00:10:53,990 INFO]  * tgt vocab size = 161817
[2020-12-09 00:10:53,990 INFO] Building model...
[2020-12-09 00:11:16,477 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(692838, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_f

[2020-12-09 00:11:16,817 INFO] Starting training on GPU: [0]
[2020-12-09 00:11:16,817 INFO] Start training loop and validate every 1000 steps...
[2020-12-09 00:11:16,818 INFO] Loading dataset from OpenNMT-py/Data/morph2morph/demo.train.0.pt
[2020-12-09 00:11:41,729 INFO] number of examples: 692803
[2020-12-09 00:12:42,843 INFO] Step 50/50000; acc:   2.64; ppl: 32860.67; xent: 10.40; lr: 0.00001; 138/4742 tok/s;     86 sec
[2020-12-09 00:13:37,812 INFO] Step 100/50000; acc:   6.20; ppl: 24471.82; xent: 10.11; lr: 0.00001; 226/7428 tok/s;    141 sec
[2020-12-09 00:14:33,077 INFO] Step 150/50000; acc:   6.26; ppl: 14950.13; xent: 9.61; lr: 0.00002; 223/7384 tok/s;    196 sec
[2020-12-09 00:15:28,376 INFO] Step 200/50000; acc:   6.16; ppl: 7636.90; xent: 8.94; lr: 0.00002; 226/7378 tok/s;    252 sec
[2020-12-09 00:16:23,744 INFO] Step 250/50000; acc:   6.25; ppl: 3566.65; xent: 8.18; lr: 0.00003; 217/7371 tok/s;    307 sec
[2020-12-09 00:17:19,100 INFO] Step 300/50000; acc:   6.30; ppl: 17

[2020-12-09 01:07:38,372 INFO] Step 2950/50000; acc:  33.78; ppl: 41.92; xent: 3.74; lr: 0.00036; 222/7360 tok/s;   3382 sec
[2020-12-09 01:08:33,914 INFO] Step 3000/50000; acc:  33.71; ppl: 41.93; xent: 3.74; lr: 0.00037; 225/7346 tok/s;   3437 sec
[2020-12-09 01:08:33,917 INFO] Loading dataset from OpenNMT-py/Data/morph2morph/demo.valid.0.pt
[2020-12-09 01:08:34,252 INFO] number of examples: 8014
[2020-12-09 01:08:51,924 INFO] Validation perplexity: 61.98
[2020-12-09 01:08:51,924 INFO] Validation accuracy: 34.5146
[2020-12-09 01:09:47,575 INFO] Step 3050/50000; acc:  33.92; ppl: 40.73; xent: 3.71; lr: 0.00038; 163/5540 tok/s;   3511 sec
[2020-12-09 01:10:43,258 INFO] Step 3100/50000; acc:  34.06; ppl: 40.11; xent: 3.69; lr: 0.00038; 215/7333 tok/s;   3566 sec
[2020-12-09 01:11:38,889 INFO] Step 3150/50000; acc:  34.06; ppl: 39.88; xent: 3.69; lr: 0.00039; 215/7333 tok/s;   3622 sec
[2020-12-09 01:12:34,162 INFO] Step 3200/50000; acc:  34.18; ppl: 39.52; xent: 3.68; lr: 0.00040; 231/7

[2020-12-09 02:02:53,645 INFO] Step 5850/50000; acc:  38.18; ppl: 26.08; xent: 3.26; lr: 0.00072; 216/7325 tok/s;   6697 sec
[2020-12-09 02:03:49,434 INFO] Step 5900/50000; acc:  38.32; ppl: 25.82; xent: 3.25; lr: 0.00073; 215/7319 tok/s;   6753 sec
[2020-12-09 02:04:45,214 INFO] Step 5950/50000; acc:  38.23; ppl: 25.85; xent: 3.25; lr: 0.00074; 215/7313 tok/s;   6808 sec
[2020-12-09 02:05:40,652 INFO] Step 6000/50000; acc:  38.46; ppl: 25.54; xent: 3.24; lr: 0.00074; 231/7364 tok/s;   6864 sec
[2020-12-09 02:05:40,655 INFO] Loading dataset from OpenNMT-py/Data/morph2morph/demo.valid.0.pt
[2020-12-09 02:05:41,852 INFO] number of examples: 8014
[2020-12-09 02:05:59,573 INFO] Validation perplexity: 46.076
[2020-12-09 02:05:59,573 INFO] Validation accuracy: 36.9468
[2020-12-09 02:06:54,993 INFO] Step 6050/50000; acc:  38.59; ppl: 25.48; xent: 3.24; lr: 0.00075; 170/5485 tok/s;   6938 sec
[2020-12-09 02:07:50,460 INFO] Step 6100/50000; acc:  38.54; ppl: 25.25; xent: 3.23; lr: 0.00075; 225/

[2020-12-09 02:58:17,847 INFO] Step 8750/50000; acc:  42.12; ppl: 18.55; xent: 2.92; lr: 0.00094; 214/7285 tok/s;  10021 sec
[2020-12-09 02:59:13,546 INFO] Step 8800/50000; acc:  42.42; ppl: 18.17; xent: 2.90; lr: 0.00094; 230/7330 tok/s;  10077 sec
[2020-12-09 03:00:09,445 INFO] Step 8850/50000; acc:  42.49; ppl: 18.11; xent: 2.90; lr: 0.00094; 226/7294 tok/s;  10133 sec
[2020-12-09 03:01:05,104 INFO] Step 8900/50000; acc:  42.51; ppl: 17.83; xent: 2.88; lr: 0.00094; 224/7332 tok/s;  10188 sec
[2020-12-09 03:02:00,722 INFO] Step 8950/50000; acc:  42.73; ppl: 17.73; xent: 2.88; lr: 0.00093; 218/7338 tok/s;  10244 sec
[2020-12-09 03:02:56,217 INFO] Step 9000/50000; acc:  42.94; ppl: 17.41; xent: 2.86; lr: 0.00093; 226/7355 tok/s;  10299 sec
[2020-12-09 03:02:56,220 INFO] Loading dataset from OpenNMT-py/Data/morph2morph/demo.valid.0.pt
[2020-12-09 03:02:58,232 INFO] number of examples: 8014
[2020-12-09 03:03:16,046 INFO] Validation perplexity: 43.788
[2020-12-09 03:03:16,046 INFO] Valida

...

[2020-12-09 12:14:40,116 INFO] Step 37750/50000; acc:  70.13; ppl:  3.16; xent: 1.15; lr: 0.00045; 223/7373 tok/s;  43403 sec
[2020-12-09 12:15:35,753 INFO] Step 37800/50000; acc:  70.21; ppl:  3.16; xent: 1.15; lr: 0.00045; 217/7336 tok/s;  43459 sec
[2020-12-09 12:16:31,507 INFO] Step 37850/50000; acc:  69.80; ppl:  3.22; xent: 1.17; lr: 0.00045; 218/7281 tok/s;  43515 sec
[2020-12-09 12:17:27,338 INFO] Step 37900/50000; acc:  70.47; ppl:  3.14; xent: 1.15; lr: 0.00045; 220/7308 tok/s;  43571 sec
[2020-12-09 12:18:22,704 INFO] Step 37950/50000; acc:  71.42; ppl:  3.04; xent: 1.11; lr: 0.00045; 242/7374 tok/s;  43626 sec
[2020-12-09 12:19:18,205 INFO] Step 38000/50000; acc:  70.84; ppl:  3.10; xent: 1.13; lr: 0.00045; 225/7354 tok/s;  43681 sec
[2020-12-09 12:19:18,207 INFO] Loading dataset from OpenNMT-py/Data/morph2morph/demo.valid.0.pt
[2020-12-09 12:19:18,548 INFO] number of examples: 8014
[2020-12-09 12:19:36,250 INFO] Validation perplexity: 219.486
[2020-12-09 12:19:36,250 INFO]

[2020-12-09 13:10:10,274 INFO] Step 40600/50000; acc:  71.42; ppl:  3.00; xent: 1.10; lr: 0.00044; 217/7347 tok/s;  46733 sec
[2020-12-09 13:11:05,866 INFO] Step 40650/50000; acc:  70.97; ppl:  3.05; xent: 1.12; lr: 0.00044; 219/7302 tok/s;  46789 sec
[2020-12-09 13:12:01,661 INFO] Step 40700/50000; acc:  71.62; ppl:  2.98; xent: 1.09; lr: 0.00044; 220/7313 tok/s;  46845 sec
[2020-12-09 13:12:57,012 INFO] Step 40750/50000; acc:  72.67; ppl:  2.88; xent: 1.06; lr: 0.00044; 242/7376 tok/s;  46900 sec
[2020-12-09 13:13:52,542 INFO] Step 40800/50000; acc:  72.08; ppl:  2.95; xent: 1.08; lr: 0.00044; 225/7351 tok/s;  46956 sec
[2020-12-09 13:14:47,915 INFO] Step 40850/50000; acc:  72.36; ppl:  2.90; xent: 1.07; lr: 0.00044; 231/7368 tok/s;  47011 sec
[2020-12-09 13:15:43,199 INFO] Step 40900/50000; acc:  72.21; ppl:  2.92; xent: 1.07; lr: 0.00044; 231/7385 tok/s;  47066 sec
[2020-12-09 13:16:38,492 INFO] Step 40950/50000; acc:  72.59; ppl:  2.89; xent: 1.06; lr: 0.00044; 234/7382 tok/s;  47

[2020-12-09 14:05:32,501 INFO] Step 43500/50000; acc:  72.58; ppl:  2.85; xent: 1.05; lr: 0.00042; 221/7330 tok/s;  50056 sec
[2020-12-09 14:06:27,873 INFO] Step 43550/50000; acc:  73.76; ppl:  2.76; xent: 1.01; lr: 0.00042; 242/7373 tok/s;  50111 sec
[2020-12-09 14:07:23,385 INFO] Step 43600/50000; acc:  72.86; ppl:  2.83; xent: 1.04; lr: 0.00042; 225/7353 tok/s;  50167 sec
[2020-12-09 14:08:18,732 INFO] Step 43650/50000; acc:  73.43; ppl:  2.77; xent: 1.02; lr: 0.00042; 231/7371 tok/s;  50222 sec
[2020-12-09 14:09:14,125 INFO] Step 43700/50000; acc:  73.33; ppl:  2.79; xent: 1.02; lr: 0.00042; 231/7370 tok/s;  50277 sec
[2020-12-09 14:10:09,503 INFO] Step 43750/50000; acc:  73.60; ppl:  2.77; xent: 1.02; lr: 0.00042; 234/7370 tok/s;  50333 sec
[2020-12-09 14:11:04,877 INFO] Step 43800/50000; acc:  73.20; ppl:  2.80; xent: 1.03; lr: 0.00042; 231/7370 tok/s;  50388 sec
[2020-12-09 14:12:00,360 INFO] Step 43850/50000; acc:  73.52; ppl:  2.77; xent: 1.02; lr: 0.00042; 228/7356 tok/s;  50

[2020-12-09 15:01:08,408 INFO] Step 46400/50000; acc:  73.72; ppl:  2.74; xent: 1.01; lr: 0.00041; 225/7356 tok/s;  53392 sec
[2020-12-09 15:02:03,683 INFO] Step 46450/50000; acc:  74.29; ppl:  2.67; xent: 0.98; lr: 0.00041; 231/7381 tok/s;  53447 sec
[2020-12-09 15:02:58,961 INFO] Step 46500/50000; acc:  74.09; ppl:  2.69; xent: 0.99; lr: 0.00041; 231/7386 tok/s;  53502 sec
[2020-12-09 15:03:54,224 INFO] Step 46550/50000; acc:  74.51; ppl:  2.66; xent: 0.98; lr: 0.00041; 235/7385 tok/s;  53557 sec
[2020-12-09 15:04:49,594 INFO] Step 46600/50000; acc:  74.01; ppl:  2.70; xent: 0.99; lr: 0.00041; 231/7371 tok/s;  53613 sec
[2020-12-09 15:05:44,923 INFO] Step 46650/50000; acc:  74.36; ppl:  2.67; xent: 0.98; lr: 0.00041; 229/7377 tok/s;  53668 sec
[2020-12-09 15:06:40,321 INFO] Step 46700/50000; acc:  74.08; ppl:  2.71; xent: 1.00; lr: 0.00041; 228/7367 tok/s;  53724 sec
[2020-12-09 15:07:35,610 INFO] Step 46750/50000; acc:  74.20; ppl:  2.69; xent: 0.99; lr: 0.00041; 227/7379 tok/s;  53

[2020-12-09 15:56:24,489 INFO] Step 49300/50000; acc:  74.96; ppl:  2.60; xent: 0.96; lr: 0.00040; 232/7387 tok/s;  56708 sec
[2020-12-09 15:57:19,801 INFO] Step 49350/50000; acc:  75.36; ppl:  2.57; xent: 0.94; lr: 0.00040; 234/7379 tok/s;  56763 sec
[2020-12-09 15:58:15,132 INFO] Step 49400/50000; acc:  74.77; ppl:  2.61; xent: 0.96; lr: 0.00040; 232/7376 tok/s;  56818 sec
[2020-12-09 15:59:10,536 INFO] Step 49450/50000; acc:  75.15; ppl:  2.59; xent: 0.95; lr: 0.00040; 229/7367 tok/s;  56874 sec
[2020-12-09 16:00:05,886 INFO] Step 49500/50000; acc:  74.86; ppl:  2.61; xent: 0.96; lr: 0.00040; 228/7373 tok/s;  56929 sec
[2020-12-09 16:01:01,172 INFO] Step 49550/50000; acc:  75.01; ppl:  2.60; xent: 0.96; lr: 0.00040; 227/7380 tok/s;  56984 sec
[2020-12-09 16:01:56,829 INFO] Step 49600/50000; acc:  74.34; ppl:  2.64; xent: 0.97; lr: 0.00040; 219/7333 tok/s;  57040 sec
[2020-12-09 16:02:52,878 INFO] Step 49650/50000; acc:  74.13; ppl:  2.67; xent: 0.98; lr: 0.00040; 215/7278 tok/s;  57

# **Translate**

Now that you have your model, you can start translating.

-model ==> Setting your model

Output predictions into pred.txt

In [ ]:
!python OpenNMT-py/translate.py -model OpenNMT-py/Data/morph2morph/model/model_step_50000.pt -src OpenNMT-py/Data/src-test-morph.txt -output OpenNMT-py/Data/morph2morph/pred_50k.txt -replace_unk -verbose -beam_size 1 -gpu 0


# Detokenization

Even after the translation process is finished, it is still in a segment, so it is different from the actual sentence structure used by real people. Thus, when you perform a detoxification process, it is returned in the form of the actual sentence.

We Use "sed" for BPE Detokenization


In [7]:
!sed -i "s/@@ //g"  OpenNMT-py/Data/pred.txt

# Post-processing

In [6]:
# f = open('OpenNMT-py/Data/tgt-test-char.txt', 'r', encoding='utf-8')
# txt = f.read()
# txt = txt.replace(' ', '')
# txt = txt.replace('ᴥ', ' ')

# with open("OpenNMT-py/Data/tgt-test-char1.txt", "w") as f:
#     f.write(txt)
    
f = open('OpenNMT-py/Data/tgt-test-morph.txt', 'r', encoding='utf-8')
txt = f.read()
txt[:1000]

'" According to descriptions by the police , Mr. Kim , who visited the PC - bang as a guest , had an argument with Mr. Shin while demanding that food left by other guests be cleared from his seat . " \n" As Jamsil Girls \' High School ( Principal Kim In - bong ) is aiming to cultivate talents and good personality , the teachers and students are upgrading their career path programs through the " " Team - play of Trust " " . " \nFormer lawmaker Kim will lose both his right to vote and his right to run for 5 years if a fine of 1 million won or more is confirmed . \n" The working environment of Daejeon Industrial Park in Daedeok - gu , Daejeon will be improved . " \n" Around 90 t of reinforcement facilities were lost due to high waves at the Dongbang Breakwater Construction site in Wimi Port , Namwon - eup , Seogwipo - si , and a 100 m median strip fell in the area of Nohyeong Rotary - Samsung Service Center in Jeju . " \nIt is a program to make students value physical education and art . 

In [2]:
f = open('OpenNMT-py/Data/morph2morph/pred_10k.txt', 'r', encoding='utf-8')
txt = f.read()
# txt = txt.replace(' ', '')
# txt = txt.replace('ᴥ', ' ')

with open("OpenNMT-py/Data/morph2morph/pred_10k-edited.txt", "w") as f:
    f.write(txt)

f = open('OpenNMT-py/Data/morph2morph/pred_10k-edited.txt', 'r', encoding='utf-8')
txt = f.read()
txt[:1000]  

'" The Ministry of Land , Infrastructure and Transport said that it will conduct a survey on the number of rental businesses in the first half of this year , and will conduct a survey on the number of rental businesses in the second half of this year . "\n" The Ministry of Land , Infrastructure and Transport said that it will conduct a survey on the number of rental businesses in the first half of this year , and will conduct a survey on the number of rental businesses in the second half of this year . "\n" The Ministry of Land , Infrastructure and Transport said that it will conduct a survey on the number of rental businesses in the first half of this year , and will conduct a survey on the number of rental businesses in the second half of this year . "\n" The Ministry of Land , Infrastructure and Transport said that it will conduct a survey on the number of rental businesses in the first half of this year , and will conduct a survey on the number of rental businesses in the second ha

# Evaluation Using BLEU

Quantitative evaluation is performed on the sentence thus obtained. BLEU is a quantitative evaluation method for machine translation. You can see which model is superior by comparing it to the BLEU score you are comparing.

https://www.aclweb.org/anthology/P02-1040

In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl OpenNMT-py/Data/tgt-test.txt < OpenNMT-py/Data/morph2morph/pred_50k.txt